In [1]:
import sys
sys.path.append("..")
import spark_utils
spark_utils.print_ui_links()

NameNode: http://ec2-3-235-150-104.compute-1.amazonaws.com:50070
YARN: http://ec2-3-235-150-104.compute-1.amazonaws.com:8088
Spark UI: http://ec2-3-235-150-104.compute-1.amazonaws.com:20888/proxy/application_1620307932889_0002


# HDFS config

https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml

In [2]:
! cat /etc/hadoop/conf/hdfs-site.xml | egrep -A 1 ".dir|.replication"

    <name>dfs.datanode.data.dir</name>
    <value>file:///mnt/hdfs</value>
--
    <name>dfs.namenode.name.dir</name>
    <value>file:///mnt/namenode</value>
--
    <name>dfs.namenode.replication.max-streams</name>
    <value>20</value>
--
    <name>dfs.namenode.replication.max-streams-hard-limit</name>
    <value>40</value>
--
    <name>dfs.namenode.replication.work.multiplier.per.iteration</name>
    <value>10</value>
--
    <name>dfs.replication</name>
    <value>1</value>
--
    <name>dfs.name.dir</name>
    <value>/mnt/namenode</value>
--
    <name>dfs.data.dir</name>
    <value>/mnt/hdfs</value>


# HDFS commands

In [3]:
! hadoop fs

Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-x] <path> ...]
	[-expunge]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] [-skip-empty-file] <src> <localdst>]
	[-help [cmd ...]]
	[-ls [-C] [-d] [-h] [-q] [-R] [-t] [-S] [-r] [-u] [<path> ...]]
	[-mkdir [-p] <path> ...]
	[-mov

# Copy files from/to HDFS

In [4]:
! echo "Test file" > test.txt

In [5]:
! ls -l .

total 68
-rw-rw-r-- 1 hadoop hadoop   149 May  6 21:23 generate.py
-rw-rw-r-- 1 hadoop hadoop 24840 May  6 21:52 hdfs-basics.ipynb
-rw-rw-r-- 1 hadoop hadoop 27369 May  6 21:12 mapreduce-wordcount.ipynb
-rw-r--r-- 1 hadoop hadoop    10 May  6 21:18 test2.txt
-rw-rw-r-- 1 hadoop hadoop    10 May  6 21:53 test.txt


In [6]:
! hadoop fs -copyFromLocal test.txt /

copyFromLocal: `/test.txt': File exists


In [7]:
! hadoop fs -ls /

Found 7 items
-rw-r--r--   1 hadoop hadoop  134217721 2021-05-06 21:24 /128.txt
-rw-r--r--   1 hadoop hadoop  268435451 2021-05-06 21:24 /256.txt
-rw-r--r--   2 hadoop hadoop         10 2021-05-06 21:18 /test.txt
-rw-r--r--   1 hadoop hadoop          8 2021-05-06 21:19 /test3.txt
drwxrwxrwt   - hdfs   hadoop          0 2021-05-06 13:32 /tmp
drwxr-xr-x   - hdfs   hadoop          0 2021-05-06 13:32 /user
drwxr-xr-x   - hdfs   hadoop          0 2021-05-06 13:32 /var


In [8]:
! hadoop fs -copyToLocal /test.txt test2.txt

copyToLocal: `test2.txt': File exists


In [9]:
! cat test2.txt

Test file


# Streaming from/to HDFS

In [10]:
! hadoop fs -cat /test.txt | wc -w

2


In [11]:
! echo "1 2 3 4" | hadoop fs -copyFromLocal - /test3.txt

copyFromLocal: `/test3.txt': File exists


In [12]:
! hadoop fs -cat /test3.txt

1 2 3 4


# Replication factor

In [13]:
! cat /etc/hadoop/conf/hdfs-site.xml | grep -A 1 "dfs.replication"

    <name>dfs.replication</name>
    <value>1</value>


In [14]:
! hadoop fs -stat %r /test.txt

2


In [15]:
! hadoop fs -setrep 2 /test.txt

Replication 2 set: /test.txt


In [16]:
! hadoop fs -stat %r /test.txt

2


# HDFS block size

In [18]:
# default block size is 128 MB
! cat /etc/hadoop/conf/hdfs-site.xml | grep -A 1 "block"

In [19]:
%%file generate.py
import sys

megabytes = int(sys.argv[1])
string = "{}M\n".format(megabytes)
count = 1024 * 1024 * megabytes // len(string) - 1
print(string * count)

Overwriting generate.py


In [20]:
! python3 ./generate.py 1 | head -n 5

1M
1M
1M
1M
1M
Traceback (most recent call last):
  File "./generate.py", line 6, in <module>
    print(string * count)
IOError: [Errno 32] Broken pipe


In [21]:
! hadoop fs -rm /128.txt /256.txt

Deleted /128.txt
Deleted /256.txt


In [22]:
! python3 ./generate.py 128 | hadoop fs -copyFromLocal - /128.txt

! python3 ./generate.py 256 | hadoop fs -copyFromLocal - /256.txt

In [23]:
! hdfs fsck / -files -blocks

Connecting to namenode via http://ip-172-31-13-255.ec2.internal:50070/fsck?ugi=hadoop&files=1&blocks=1&path=%2F
FSCK started by hadoop (auth:SIMPLE) from /172.31.13.255 for path / at Thu May 06 21:54:23 UTC 2021
/ <dir>
/128.txt 134217721 bytes, 1 block(s):  OK
0. BP-1596529054-172.31.13.255-1620307919098:blk_1073741851_1027 len=134217721 Live_repl=1

/256.txt 268435451 bytes, 2 block(s):  OK
0. BP-1596529054-172.31.13.255-1620307919098:blk_1073741852_1028 len=134217728 Live_repl=1
1. BP-1596529054-172.31.13.255-1620307919098:blk_1073741853_1029 len=134217723 Live_repl=1

/test.txt 10 bytes, 1 block(s):  OK
0. BP-1596529054-172.31.13.255-1620307919098:blk_1073741846_1022 len=10 Live_repl=2

/test3.txt 8 bytes, 1 block(s):  OK
0. BP-1596529054-172.31.13.255-1620307919098:blk_1073741847_1023 len=8 Live_repl=1

/tmp <dir>
/tmp/hadoop-yarn <dir>
/tmp/hadoop-yarn/staging <dir>
/tmp/hadoop-yarn/staging/history <dir>
/tmp/hadoop-yarn/staging/history/done <dir>
/tmp/hadoop-yarn/staging/history

# How HDFS looks like on disk (on remote nodes)

In [24]:
# first copy ~/andrey.pem (your ssh key) to master node by running on your machine:
print(spark_utils.copy_ssh_key_hint())

scp -i ~/andrey.pem ~/andrey.pem hadoop@ec2-3-235-150-104.compute-1.amazonaws.com:~


In [25]:
# list all nodes
! yarn node -list

21/05/06 21:54:43 INFO client.RMProxy: Connecting to ResourceManager at ip-172-31-13-255.ec2.internal/172.31.13.255:8032
Total Nodes:2
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
ip-172-31-13-173.ec2.internal:8041	        RUNNING	ip-172-31-13-173.ec2.internal:8042	                           1
ip-172-31-4-27.ec2.internal:8041	        RUNNING	ip-172-31-4-27.ec2.internal:8042	                           0


In [28]:
! ssh -i ~/andrey.pem -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null hadoop@ip-172-31-13-173.ec2.internal "sudo bash -c 'ls -lh -R /mnt/hdfs'"

/mnt/hdfs:
total 4.0K
drwxr-xr-x 3 hdfs hdfs 70 May  6 13:32 current
-rw-r--r-- 1 hdfs hdfs 34 May  6 13:32 in_use.lock

/mnt/hdfs/current:
total 4.0K
drwx------ 4 hdfs hdfs  54 May  6 13:32 BP-1596529054-172.31.13.255-1620307919098
-rw-r--r-- 1 hdfs hdfs 229 May  6 13:32 VERSION

/mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098:
total 4.0K
drwxr-xr-x 4 hdfs hdfs  49 May  6 13:32 current
-rw-r--r-- 1 hdfs hdfs 166 May  6 13:32 scanner.cursor
drwxr-xr-x 2 hdfs hdfs   6 May  6 21:21 tmp

/mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current:
total 4.0K
drwxr-xr-x 3 hdfs hdfs  21 May  6 13:42 finalized
drwxr-xr-x 2 hdfs hdfs   6 May  6 21:54 rbw
-rw-r--r-- 1 hdfs hdfs 145 May  6 13:32 VERSION

/mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized:
total 0
drwxr-xr-x 3 hdfs hdfs 21 May  6 13:42 subdir0

/mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0:
total 4.0K
drwxr-xr-x 2 hdfs hdfs 4.0K May  6 21:54 s

In [29]:
! ssh -i ~/andrey.pem -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null hadoop@ip-172-31-13-173.ec2.internal "sudo bash -c 'find /mnt/hdfs | grep blk | xargs head -c 32'"

==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0/subdir0/blk_1073741838_1014.meta <==
    f��wb�
,��X�(C	�Y1��"
==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0/subdir0/blk_1073741838 <==
��h��׶9�A@���P	 VERSION   
==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0/subdir0/blk_1073741840_1016.meta <==
    �u�*��k�4���w�J@;z8�!�
==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0/subdir0/blk_1073741840 <==
PK  �v�R               ja
==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0/subdir0/blk_1073741841_1017.meta <==
    �T����Q�Vt�/*�����հA=�
==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/finalized/subdir0/subdir0/blk_1073741841 <==
�ؓrWJ����(���е]�>�ȳ�A��
==> /mnt/hdfs/current/BP-1596529054-172.31.13.255-1620307919098/current/fina